In [1]:
import PyPDF2

In [2]:
from typing import Any
from langchain import LLMChain

import openai
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import re

In [3]:
from typing import Dict
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema.output import LLMResult
class MyCustomHandler(BaseCallbackHandler):
    def on_text(self, text: str, **kwargs: Any) -> Any:
        print(f"Text: {text}")
        self.log = text

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        print(response.llm_output.keys())
        print(response.llm_output)
"""Chain of Density prompting: https://arxiv.org/abs/2309.04269"""
user_sum_prompt = """
Article: {text}
You will generate increasingly concise entity-dense summaries of the above article. Repeat the following 2 steps 5 times.

Step 1: Identify 1-3 informative entities (delimited) from the article which are missing from the previously generated summary.
Step 2: Write a new denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.

A missing entity is
- Relevant: to the main stories.
- Specific: descriptive yet concise (5 words or fewer).
- Novel: not in the previous summary.
- Faithful: present in the article.
- Anywhere: located in the article.

Guidelines:
- The first summary should be long (4-5 sentences, ~80 words), yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
- Make every word count. Rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise, yet self-contained, e.g., easily understood without the article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Remember: Use the exact same number of words for each summary.

Do not include any explanations. Only return a valid valid RFC8259 compilant JSON.
The JSON should be an array of length 5 following this format:
{{
"data": [
   {{
      "missing_entities":  "An array of missing entitys"
      "denser_summary": "denser summary, covers every entity in detail"
   }}
]
}}
The response in JSON format:
"""

user_translate_prompt = """
Übersetze das folgende JSON in {language}. Beinhalte die Formatierung als RFC8259 JSON bei.
Das JSON sollte ein Array der Länge 5 sein, welcher folgendem Format folgt:
{{
"data": [
   {{
      "missing_entities": "An array of missing entitys"
      "denser_summary": "denser summary, covers every entity in detail"
   }}
]
}}

JSON: {sum}
"""
def getSummarizationPrompt() -> PromptTemplate:
    return PromptTemplate(input_variables=["text"], template=user_sum_prompt)

def getTranslationPrompt() -> PromptTemplate:
    return PromptTemplate(input_variables=["language", "sum"], template=user_translate_prompt)
import os
os.environ['http_proxy'] = "http://internet-proxy-client.muenchen.de:80" 
os.environ['https_proxy'] = "http://internet-proxy-client.muenchen.de:80" 
openai.api_type = "azure"
openai.api_base = f"https://cog-tb726faeoukc2.openai.azure.com"
openai.api_version = "2023-05-15"
openai.api_key = "9324d0a0d0bd4d72b0e34596a9dc4a9c"
verbose = False
llm = AzureChatOpenAI(
    model="gpt-35-turbo",
    temperature= 0.7,
    max_tokens=1000,
    n=1,
    deployment_name= "chat",
    openai_api_key=openai.api_key,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    openai_api_type=openai.api_type,
    callbacks=[MyCustomHandler()]
)
summarizationChain = LLMChain(llm=llm, prompt=getSummarizationPrompt(), output_key="sum", verbose=verbose)
translationChain = LLMChain(llm=llm, prompt=getTranslationPrompt(), output_key="translation", verbose=verbose)
overall_chain = SequentialChain(
    chains=[summarizationChain, translationChain], 
    input_variables=["language", "text"],
    output_variables=["translation", "sum"],
    verbose=verbose
)



C:\Users\michael.jaumann\AppData\Roaming\Python\Python38\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [20]:
from langchain.text_splitter import TokenTextSplitter
from langchain.document_loaders import PyPDFLoader
def readPDF(pdf, chunk_size=2500, chunk_overlap=100):
    # creating a pdf reader object
    reader = PyPDF2.PdfReader(pdf)
    # read alll
    complete = ""
    for page in reader.pages:
        complete += page.extract_text()
    #split
    text_splitter = TokenTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_text(complete)
    return texts

def readPDFLangchain(pdf, chunk_size=2500, chunk_overlap=100):
     # creating a pdf reader object
    loader = PyPDFLoader(pdf)
    # read alll
    pages = loader.load()
    #split
    text_splitter = TokenTextSplitter.from_tiktoken_encoder(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(pages)
    return texts
    

texts = readPDF("roemische-geschichte-100-50vchr.pdf",chunk_size=1500, chunk_overlap=100)
language = "german"

In [21]:
len(texts)

6

In [6]:
from langchain_core.runnables import RunnableParallel

In [30]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.combine_documents.stuff import StuffDocumentsChain


# Map
map_template = """The following is a set of documents
{docs}
Can you provide a comprehensive summary of the given set of documents? The summary should cover all the key points and main ideas presented in the original text, while also condensing the information into a concise and easy-to-understand format. Please ensure that the summary includes relevant details and examples that support the main ideas, while avoiding any unnecessary information or repetition. The length of the summary should be appropriate for the length and complexity of the original text, providing a clear and accurate overview without omitting any important information
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=summarizationChain, document_variable_name="text"
)

reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
    output_key="sum"
)
overall_chain = SequentialChain(
    chains=[map_reduce_chain, translationChain], 
    input_variables=["language", "input_documents"],
    output_variables=["translation", "sum"])

texts_langchain = readPDFLangchain("roemische-geschichte-100-50vchr.pdf",chunk_size=2000, chunk_overlap=100)
results = overall_chain({'language':language, 'input_documents': texts_langchain})
#results = translationChain({'language':language, 'sum': map_reduce_chain.run(texts_langchain)})

dict_keys(['token_usage', 'model_name', 'system_fingerprint'])
{'token_usage': <OpenAIObject at 0x1f39c1e7f40> JSON: {
  "prompt_tokens": 740,
  "completion_tokens": 184,
  "total_tokens": 924
}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': ''}
dict_keys(['token_usage', 'model_name', 'system_fingerprint'])
{'token_usage': <OpenAIObject at 0x1f39c1e7d10> JSON: {
  "prompt_tokens": 998,
  "completion_tokens": 187,
  "total_tokens": 1185
}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': ''}
dict_keys(['token_usage', 'model_name', 'system_fingerprint'])
{'token_usage': <OpenAIObject at 0x1f39c1e7220> JSON: {
  "prompt_tokens": 893,
  "completion_tokens": 188,
  "total_tokens": 1081
}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': ''}
dict_keys(['token_usage', 'model_name', 'system_fingerprint'])
{'token_usage': <OpenAIObject at 0x1f39c1e7900> JSON: {
  "prompt_tokens": 831,
  "completion_tokens": 89,
  "total_tokens": 920
}, 'model_name': 'gpt-35-turbo', 'system_fingerpri

In [31]:
results.keys()

dict_keys(['language', 'input_documents', 'translation', 'sum'])

In [32]:
results['input_documents']

[Document(page_content='Geschichte der römischen Literatur\nund Gesellschaft\nEine tabellarische Übersicht zur römischen Geschichte und\nLiteraturgeschichte von den Anfängen bis in die Spätantike\n(400 n. Chr.)\nDokument 2: Von 100 v. Chr. bis 50 v. Chr. - Der Kampf um die \nRepublik\nWeitere Dokumente: \n•Tabelle zur römischen Geschichte – vollständig, HTML, in den Seiten des \nLandesbildungsservers: \nwww.schule-bw.de/faecher-und-schularten/sprachen-und-literatur/latein/antike-kultur/\nroemische-geschichte/daten/roemische-geschichte-tabelle.html  \n•Download der Dokumente in den Formaten PDF, WORD und OpenOffice: www.schule-\nbw.de/faecher-und-schularten/sprachen-und-literatur/latein/antike-kultur/roemische-\ngeschichte/downloadbereich  \n•Die Daten, die in dieser Tabelle aufgelistet sind, entsprechen denen der interaktiven Zeitleiste \nzur römischen Geschichte und zur antiken Philosophie . \n•Methodische und didaktische Hinweise   für Lehrkräfte \n•Interaktive Übungen   \n•Hinweise 

In [33]:
results['translation']

'{\n   "data":[\n      {\n         "missing_entities":[\n            "Spartacus",\n            "Crassus",\n            "Slave Revolt"\n         ],\n         "denser_summary":"Die Dokumente umfassen vier wichtige Ereignisse in der antiken römischen Geschichte. Der Sklavenaufstand unter Führung von Spartacus in den Jahren 73-71 v. Chr. war eine große Bedrohung für das Römische Reich, wobei der ehemalige Gladiator eine große Armee von Sklaven versammelte, bevor er von Crassus und Pompey besiegt wurde. Der Aufstand von Sertorius in den Jahren 79-72 v. Chr. sah einen abtrünnigen Offizier eine rivalisierende Senatsversammlung auf der iberischen Halbinsel gründen, nur um nach internen Konflikten durch Pompey und Metellus Pius besiegt zu werden, was zur Ermordung von Sertorius führte. Der Prozess gegen Verres im Jahr 70 v. Chr. sah Cicero als Roms bekanntesten Anwalt zu Ruhm gelangen, indem er erfolgreich den korrupten Gouverneur von Sizilien anklagte. Schließlich beinhaltete die Verschwörung 

In [7]:
texts_langchain

NameError: name 'texts_langchain' is not defined

In [11]:
text_splitter = TokenTextSplitter.from_tiktoken_encoder(
        chunk_size=400, chunk_overlap=0)
texts = text_splitter.split_text("Viele wollen es. 57 Prozent der Deutschen sagen in einer Umfrage des Meinungsforschungsinstituts YouGov, dass sie für ein Tempolimit von 130 Stundenkilometern auf Autobahnen sind. Genauso sind es Menschen aus Wissenschaft, Politik, Kirche und dem öffentlichen Leben, die im Bündnis Tempolimit jetzt! aktiv sind. Einer davon: Sebastian Vettel, der vier Saisons der Formel-1 gewonnen hat.​Einer, der es definitiv nicht will, ist Verkehrsminister Volker Wissing. Der Politiker aus der Freien Demokratischen Partei (FDP) ist gegen staatliche Limitierungen im Straßenverkehr. Er sagt: „Autofahren bedeutet Freiheit.“ Mit dieser Meinung ist Wissing in seiner Partei und in Deutschland nicht allein.​Fast allein ist das Land aber mit der Möglichkeit, auf Autobahnen so schnell zu fahren, wie man will. In Europa hat nur ein anderer Ort kein Limit: die Isle of Man in der Irischen See. Autobahnen gibt es dort aber keine. Auch insgesamt gibt es nur wenige andere Länder ohne Maximalgeschwindigkeit, sie haben aber schlechte Straßen. ​Über ein Tempolimit diskutiert man in Deutschland schon ziemlich lang – und meistens emotional. Dabei ist 1953 ein wichtiges Jahr. Damals fand unter Kanzler Konrad Adenauer eine große Deregulierung der Geschwindigkeit statt: Menschen in Pkw und auf Motorrädern durften überall in der Bundesrepublik so schnell fahren, wie sie wollten und konnten. Westdeutschland lebte seinen Mythos der Autobahn. Anders in der Deutschen Demokratischen Republik: Dort waren 100 Stundenkilometer auf Autobahnen das Maximum, 80 außerhalb von Orten und 50 in Orten.​Weil in der Bundesrepublik die Zahl der Verkehrstoten deutlich stieg, führte die Regierung im Jahr 1957 für Orte eine Maximalgeschwindigkeit von 50 Stundenkilometern ein. Diese Norm hatte viele Gegner: den Allgemeinen Deutschen Automobil-Club (ADAC), die Autoindustrie, aber auch Menschen aus Politik und Wissenschaft.​Im Jahr 1972 kam die Einführung eines Tempolimits von 100 Stundenkilometern auf Landstraßen. Auch über diese Aktion gegen die vielen Verkehrstoten gab es intensive Diskussionen. ​Monate später kam die Ölkrise – und mit ihr ab November 1973 ein vorübergehendes Tempolimit von 100 Stundenkilometern auf Autobahnen. So wollte man den Erdölverbrauch reduzieren. Aber der Protest war groß. Ganz vorn dabei war der ADAC, der eine Kampagne mit dem Motto „Freie Fahrt für freie Bürger“ startete. Mit Erfolg: Nach 111 Tagen endete das Tempolimit auf Autobahnen. Ab dem 15. März 1974 durfte man wieder so viel Gas geben, wie man wollte. Das ist bis heute auf circa 70 Prozent der deutschen Autobahnen möglich. Dort gibt es nur die Empfehlung für eine Geschwindigkeit von 130 Stundenkilometern. ​ Die Debatte über das Tempolimit hat trotzdem nicht aufgehört. Und auch 50 Jahre später spielen die mächtige deutsche Autolobby und das (oft auch von ihr verwendete) Freiheitsargument noch immer zentrale Rollen – obwohl die Welt heute eine andere ist.​ Da ist vor allem die Frage, ob ein Tempolimit gegen die Klimakrise hilft. Anfang dieses Jahres hat das Umweltbundesamt eine Studie publiziert. Sie zeigt, dass ein Limit von 120 Stundenkilometern einen größeren Effekt hätte als bis jetzt gedacht. Pro Jahr könnte man die Treibhausgasemissionen des deutschen Straßenverkehrs so um 4,2 Prozent reduzieren.​ Elementar ist außerdem die Frage, ob ein Tempolimit den Verkehr sicherer macht. Klar ist: Je schneller ein Auto unterwegs ist, desto länger dauert das Bremsen bis zum Stoppen. Das Nachrichtenportal Spiegel Online hat den Unfallatlas der statistischen Ämter des Bundes und der Länder analysiert. Das Resultat: Je eine Milliarde gefahrener Kilometer gibt es auf Autobahnabschnitten mit Tempolimit 0,95 tödliche Unfälle. Auf Strecken ohne Tempolimit sind es 1,67 tödliche Unfälle – rund 75 Prozent mehr.​ Warum lieben viele Deutsche das schnelle Fahren trotzdem so sehr? Fragt man Verkehrspsychologen, hört man bald wieder das große Wort dieser Debatte: Freiheit. Aber dominiert der Wunsch danach wirklich die kollektive Psyche und entscheidet über das Tempo deutscher Autofahrerinnen?​Eine Antwort liefert eine Studie des Instituts der deutschen Wirtschaft aus dem Jahr 2021. Sie zeigt: 77 Prozent der Autofahrer sind auf Autobahnabschnitten ohne Tempolimit langsamer als 130 Stundenkilometer unterwegs. Zwölf Prozent fahren dort zwischen 130 und 140 Stundenkilometer. Nur weniger als zwei Prozent fahren schneller als 160.​Inzwischen ist auch die Mehrheit der Mitglieder des ADAC für ein Tempolimit. Der Verband der Automobilindustrie ist noch dagegen. Und die Regierung? Im Koalitionsvertrag steht auf Wunsch der FDP: „Ein generelles Tempolimit wird es nicht geben.")
from langchain.docstore.document import Document
docs = [Document(page_content=t) for t in texts]
results = overall_chain({'language':language, 'input_documents': docs})

dict_keys(['token_usage', 'model_name'])
{'token_usage': <OpenAIObject at 0x21a3e58a270> JSON: {
  "prompt_tokens": 412,
  "completion_tokens": 119,
  "total_tokens": 531
}, 'model_name': 'gpt-35-turbo'}
dict_keys(['token_usage', 'model_name'])
{'token_usage': <OpenAIObject at 0x21a3e58a900> JSON: {
  "prompt_tokens": 436,
  "completion_tokens": 137,
  "total_tokens": 573
}, 'model_name': 'gpt-35-turbo'}
dict_keys(['token_usage', 'model_name'])
{'token_usage': <OpenAIObject at 0x21a3e58a360> JSON: {
  "prompt_tokens": 415,
  "completion_tokens": 132,
  "total_tokens": 547
}, 'model_name': 'gpt-35-turbo'}
dict_keys(['token_usage', 'model_name'])
{'token_usage': <OpenAIObject at 0x21a3e58a220> JSON: {
  "prompt_tokens": 424,
  "completion_tokens": 139,
  "total_tokens": 563
}, 'model_name': 'gpt-35-turbo'}
dict_keys(['token_usage', 'model_name'])
{'token_usage': <OpenAIObject at 0x21a3e58a310> JSON: {
  "prompt_tokens": 201,
  "completion_tokens": 68,
  "total_tokens": 269
}, 'model_name

In [12]:
results['input_documents']

[Document(page_content='Viele wollen es. 57 Prozent der Deutschen sagen in einer Umfrage des Meinungsforschungsinstituts YouGov, dass sie für ein Tempolimit von 130 Stundenkilometern auf Autobahnen sind. Genauso sind es Menschen aus Wissenschaft, Politik, Kirche und dem öffentlichen Leben, die im Bündnis Tempolimit jetzt! aktiv sind. Einer davon: Sebastian Vettel, der vier Saisons der Formel-1 gewonnen hat.\u200bEiner, der es definitiv nicht will, ist Verkehrsminister Volker Wissing. Der Politiker aus der Freien Demokratischen Partei (FDP) ist gegen staatliche Limitierungen im Straßenverkehr. Er sagt: „Autofahren bedeutet Freiheit.“ Mit dieser Meinung ist Wissing in seiner Partei und in Deutschland nicht allein.\u200bFast allein ist das Land aber mit der Möglichkeit, auf Autobahnen so schnell zu fahren, wie man will. In Europa hat nur ein anderer Ort kein Limit: die Isle of Man in der Irischen See. Autobahnen gibt es dort aber keine. Auch insgesamt gibt es nur wenige andere Länder ohne

In [13]:
results['translation']

'RFC8259 JSON:\n\n{\n   "data":[\n      {\n         "missing_entities":[\n            "Verkehrsminister Volker Wissing",\n            "FDP"\n         ],\n         "denser_summary":"Die Debatte über die Einführung eines Tempolimits von 130 km/h auf deutschen Autobahnen geht weiter. Während eine Mehrheit der Deutschen und verschiedene öffentliche Persönlichkeiten ein Tempolimit befürworten, sind Verkehrsminister Volker Wissing und seine Partei, die FDP, dagegen und argumentieren, dass Autofahren ein Symbol für Freiheit ist."\n      },\n      {\n         "missing_entities":[\n            "Isle of Man"\n         ],\n         "denser_summary":"Deutschland ist eines der wenigen Länder in Europa ohne Tempolimit auf Autobahnen, nur die Isle of Man hat eine ähnliche Politik. Die Geschichte der Geschwindigkeitsbegrenzungen in Deutschland ist umstritten, wobei Befürworter höherer Geschwindigkeiten für Freiheit argumentieren und Gegner Sicherheitsbedenken anführen."\n      },\n      {\n         "m

In [8]:
def removeQuotations(st):
    # finds all denser summarys, replaces quotation inside with &quot
    m = re.finditer(r'(?<=\"denser_summary\":)(.*?)(?=\})', st)

    new_string = ""
    idx = 0

    for i in list(m):
        ss, se = i.span(1)  # first and last index
        groups = i.group()  # complete string ins
        quotations = [m.start() for m in re.finditer('"', groups)]
        # Quotation inside dense summary?
        if(len(quotations)>2):
            new_string += st[idx:ss+quotations[1]]
            idx = ss+quotations[1]+1
            for quotindex in quotations[1:-1]:
                new_string += st[idx:ss+quotindex] + "“ "
                idx = ss+quotindex+1
            new_string += st[idx:se]
            idx = quotations[-1]+1
        else:
            new_string += st[idx:ss] +  groups
        idx = se
    new_string += st[idx:]
    return new_string

In [22]:
from concurrent.futures import ThreadPoolExecutor
from random import randint
from time import sleep
from langchain.callbacks import get_openai_callback
import json

def call_and_cleanup(text, llmchain):
    with get_openai_callback() as cb:
        result = llmchain.invoke({"text": text, "language": language})
    total_tokens = cb.total_tokens
    chat_translate_result= result["sum"][result["sum"].index("{"):]
    chat_translate_result = chat_translate_result.replace("\n", "").rstrip()
    chat_translate_result = removeQuotations(chat_translate_result)
    if not chat_translate_result.endswith("}"):
        chat_translate_result  = chat_translate_result + "\"}]}"
    try:
        jsoned = json.loads(chat_translate_result)
    except Exception:
        # try again
        try: 
           (chat_translate_result, total_tokens) =  call_and_cleanup(text, llmchain)   
           return (chat_translate_result, total_tokens)
        except Exception:
            total_tokens = 0
            jsoned = { }
            jsoned['data'] = [{'missing_entities': 'Fehler','denser_summary': 'Zusammenfassung konnte nicht generiert werden. Bitte nochmals versuchen.'}]

    cleaned = []
    for (i, element) in enumerate(jsoned['data']):
        missing = element['missing_entities']
        if(isinstance(missing, str)):
            element['missing_entities'] = [missing]
        cleaned.append(element)
    return (cleaned,total_tokens)


def run_io_tasks_in_parallel(tasks):
    l = []
    with ThreadPoolExecutor() as executor:
        running_tasks = [executor.submit(task) for task in tasks]
        for running_task in running_tasks:
            l.append(running_task.result())
    return l

results = run_io_tasks_in_parallel(
    list(map(lambda x:  lambda: call_and_cleanup(x, summarizationChain), texts)))

dict_keys(['token_usage', 'model_name', 'system_fingerprint'])
{'token_usage': <OpenAIObject at 0x1f39c93b220> JSON: {
  "prompt_tokens": 1079,
  "completion_tokens": 398,
  "total_tokens": 1477
}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': ''}
dict_keys(['token_usage', 'model_name', 'system_fingerprint'])
{'token_usage': <OpenAIObject at 0x1f39bb3d400> JSON: {
  "prompt_tokens": 1573,
  "completion_tokens": 366,
  "total_tokens": 1939
}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': ''}
dict_keys(['token_usage', 'model_name', 'system_fingerprint'])
{'token_usage': <OpenAIObject at 0x1f388ed8db0> JSON: {
  "prompt_tokens": 1593,
  "completion_tokens": 485,
  "total_tokens": 2078
}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': ''}
dict_keys(['token_usage', 'model_name', 'system_fingerprint'])
{'token_usage': <OpenAIObject at 0x1f39c93bf40> JSON: {
  "prompt_tokens": 1565,
  "completion_tokens": 408,
  "total_tokens": 1973
}, 'model_name': 'gpt-35-turbo', 'system_fi

In [29]:
total_token = 0
summarys = []
for i in range(0,5):
    next_summary =   {"denser_summary": "", "missing_entities": []}
    for (result, tokens) in results:
        total_token += tokens
        next_summary["denser_summary"] += " "+ result[i]["denser_summary"]
        next_summary["missing_entities"] += result[i]["missing_entities"]
    summarys.append(next_summary)
summarys[3]

{'denser_summary': " This document covers the period from 100 BC to 50 BC and focuses on the fight for the Republic. Sulla becomes dictator, Proscriptions result in the deaths of thousands, and Cicero defends Sextus Roscius and serves as consul during the Catilinarian Conspiracy. Sergius Catilina conspired to assassinate the Roman consul, Cicero, in 63 BC after failing to win the consulship. Cicero uncovered the Catilinarian Conspiracy and revealed it in his famous speeches before the Senate and the people. Gaius Julius Caesar, a member of the populares party, was elected consul in 59 BC and formed the First Triumvirate with Marcus Licinius Crassus and Gnaeus Pompey Magnus. From 58 BC to 51 BC, he waged a successful war in Gaul. The Triumvirate was an informal alliance to shape the Roman Republic's politics, with no legal basis. Crassus, part of the Triumvirate, suffers a major defeat against the Parthian Empire at Carrhae in 53 BC, resulting in his death and the loss of thousands of R

In [51]:
user_translate_and_cleanupt = """
Übersetze den folgenden Text in {language}.

Beachte folgende Regeln:
- Vermeide allgemeine Phrasen wie "Das Dokument behandelt"
- Füge Füllwörter ein, um ihn einfacher zu lesen machen.
- Der Text sollte self-contained und einfach zu verstehen sein
JSON: {sum}
"""

def getTranslationCleanupPrompt() -> PromptTemplate:
    return PromptTemplate(input_variables=["language", "sum"], template=user_translate_and_cleanupt)

translateAndCleanupChain = LLMChain(llm=llm, prompt=getTranslationCleanupPrompt(), output_key="translation", verbose=verbose)

In [65]:
result = translateAndCleanupChain.invoke({"language": language, "sum": summarys[3]['denser_summary']})

dict_keys(['token_usage', 'model_name', 'system_fingerprint'])
{'token_usage': <OpenAIObject at 0x1f39e0c8270> JSON: {
  "prompt_tokens": 430,
  "completion_tokens": 493,
  "total_tokens": 923
}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': ''}


In [67]:
result['translation']

'Dieses Dokument behandelt den Zeitraum von 100 v. Chr. bis 50 v. Chr. und konzentriert sich auf den Kampf für die Republik. Sulla wird zum Diktator, Proskriptionen führen zum Tod von Tausenden und Cicero verteidigt Sextus Roscius und dient als Konsul während der Catilinarischen Verschwörung. Sergius Catilina plant 63 v. Chr. die Ermordung des römischen Konsuls Cicero, nachdem er die Konsulswahl verloren hat. Cicero deckt die Catilinarische Verschwörung auf und enthüllt sie in seinen berühmten Reden vor dem Senat und dem Volk. Gaius Julius Caesar, Mitglied der Populares-Partei, wird 59 v. Chr. zum Konsul gewählt und bildet mit Marcus Licinius Crassus und Gnaeus Pompey Magnus das erste Triumvirat. Von 58 v. Chr. bis 51 v. Chr. führt er einen erfolgreichen Krieg in Gallien. Das Triumvirat ist eine informelle Allianz, um die Politik der römischen Republik zu gestalten, ohne rechtliche Grundlage. Crassus erleidet 53 v. Chr. bei Carrhae eine schwere Niederlage gegen das Partherreich, was zu

In [64]:
summarys[4]

{'denser_summary': " The article provides an overview of Roman history and literature from the Republic to the Late Antiquity. It includes a table of events and a section on methodological and didactic suggestions for teachers, as well as interactive exercises and links to the curriculum. This article discusses the major events in the late Roman Republic, including the political career and military campaigns of Gnaeus Pompeius Magnus, the slave uprising led by Spartacus, the trial of Gaius Verres, and the Catilinarian Conspiracy to overthrow the Roman Republic, as well as the Sertorian War on the Iberian Peninsula. Gaius Julius Caesar, a member of the populares party, was elected consul in 59 BC and formed the First Triumvirate with Marcus Licinius Crassus and Gnaeus Pompey Magnus. From 58 BC to 51 BC, he waged a successful war in Gaul, which ended with the conquest of the region. The First Triumvirate was an informal alliance to shape the Roman Republic's politics, with no legal basis